<a href="https://colab.research.google.com/github/LucasQCosta/docker_aula/blob/main/Lucas_Costa_OFDM_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## import Python libraries
import numpy as np

# import libraries from matplotlib it can create a large variety of plots, but
# without live iteration
import matplotlib.pyplot as plt

# import libraries from plotly. It can create a fewer types of plot than
# matplotlib, but it allows live iteration
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.signal import firls, freqz, firwin

from tabulate import tabulate

# for iterative buttons, scrolls, etc
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display, HTML

# For run the code of modulation
from itertools import product
import math

# Simulation Parameters and Configurations

In [ ]:
# Use "np.random.seed(0)" to get the same random numbers and reproduce the same
# results
np.random.seed(0)

# FFT size (N limits the maximum number of Subcarriers) usually it is 2^x
N = 16

# Number of OFDM symbols
NOfdmSyms = 1000

# The Cyclic prefix in samples for each OFDM symbol
CP_length = 10 # prefixo ciclico

# Sampling Frequency (The sampling period Ts = 1/Fs)
Fs = 30.72e6

# Create a list with all subcarrier indexes
# np.arange(0, N) creates a list from 0 to N-1, i.e., all subcarriers
pilot_sub_carrier_indexes = np.arange(0, N)
# indexes of OFDM symbols with pilots
# with np.arange(0,4) means that the first 4 OFDM symbols carriers pilots
pilots_ofdmSyms_indexes = np.arange(0,4)

# the used subcarriers that transport data.
# With "sub_carriers_indexes = [1,2,3,6,7]", the
# subcarriers 1,2,3,6 and 7 are modulates with QAM symbols
sub_carriers_indexes = np.arange(0,N)

# Create a list of indexes of OFDM symbols that transport data
# list(range(4,NOfdmSyms+1)) create a list from 4 to N
ofdmSyms_with_data = np.arange(4, NOfdmSyms)

# Channel parameter
Pnoise = 1e-4

## plots
subcarriers_usage = np.zeros(N)
subcarriers_usage[sub_carriers_indexes] = 1
fig  = px.bar(y=subcarriers_usage, title="Subcarriers used to transport data")
fig.update_xaxes(title_text='Subcarrier index')
fig.update_yaxes(title_text='0 (not used) or 1 (used)')
fig.show()

In [ ]:
import numpy as np
import plotly.express as px

# Use "np.random.seed(0)" to get the same random numbers every time for reproducibility
np.random.seed(0)

# FFT size (N limits the maximum number of Subcarriers) usually it is 2^x
N = 16

# Number of OFDM symbols
NOfdmSyms = 100

# The Cyclic prefix in samples for each OFDM symbol
CP_length = 10 # prefixo ciclico

# Sampling Frequency (The sampling period Ts = 1/Fs)
Fs = 30.72e6

# Create a list with all subcarrier indexes
pilot_sub_carrier_indexes = np.arange(0, N)

# Number of OFDM data symbols between pilot symbols
pilot_interval = 3

# Calculate the indexes of OFDM symbols with pilots considering the pilot interval
pilots_ofdmSyms_indexes = np.arange(0, NOfdmSyms, pilot_interval + 1)

# The used subcarriers that transport data.
sub_carriers_indexes = np.arange(0, N)

# Create a list of indexes of OFDM symbols that transport data excluding the pilot symbols
ofdmSyms_with_data = np.setdiff1d(np.arange(NOfdmSyms), pilots_ofdmSyms_indexes)

# Channel parameter
Pnoise = 1e-4

## Plots
subcarriers_usage = np.zeros(N)
subcarriers_usage[sub_carriers_indexes] = 1

fig  = px.bar(y=subcarriers_usage, title="Subcarriers used to transport data")
fig.update_xaxes(title_text='Subcarrier index')
fig.update_yaxes(title_text='0 (not used) or 1 (used)')
fig.show()


# Constellation

In [ ]:
def binary_digits(type: int):
  caracteres = [1, 0]
  if(type>1):
    permsList = list(product(caracteres, repeat=type))
  else:
    permsList = caracteres
  return permsList

def modulation(modulation):
  constellation_symbols = []

  if modulation == 'None':
    raise ValueError("Add a modulation scheme.")
  elif modulation == "BPSK":
    binary_combinations = binary_digits(1)#[(0),(1)]
    for i in range(2**1):
      constellation_symbols.append(1/math.sqrt(2)*((1-2*binary_combinations[i]) + 1j*(1-2*binary_combinations[i])))
  elif modulation == "QPSK":
    binary_combinations = binary_digits(2)#[(0,1),(1,0),(0,0),(1,1)]
    for i in range(2**2):
      constellation_symbols.append(1/math.sqrt(2)*((1-2*binary_combinations[i][0]) + 1j*(1-2*binary_combinations[i][1])))
  elif modulation == "16QAM":
    binary_combinations = binary_digits(4)#[(0,0,0,0),(0,0,0,1),(0,0,1,0),(0,0,1,1),...]
    for i in range(2**4):
      constellation_symbols.append(1/math.sqrt(10)*(((1-2*binary_combinations[i][0])*(2-(1-2*binary_combinations[i][2])))+1j*((1-2*binary_combinations[i][1]))*(2-(1-2*binary_combinations[i][3]))))
  elif modulation == "64QAM":
    binary_combinations = binary_digits(6)
    for i in range(2**6):
      constellation_symbols.append( 1/math.sqrt(42)*(((1-2*binary_combinations[i][0])*(4-(1-2*binary_combinations[i][2])*(2-(1-2*binary_combinations[i][4]))))+1j*((1-2*binary_combinations[i][1])*(4-(1-2*binary_combinations[i][3])*(2-(1-2*binary_combinations[i][5]))))))
  else: # "256QAM"
    binary_combinations = binary_digits(8)
    for i in range(2**8):
        constellation_symbols.append(1/math.sqrt(170)*(((1-2*binary_combinations[i][0])*(8-(1-2*binary_combinations[i][2])*(4-(1-2*binary_combinations[i][4])*(2-(1-2*binary_combinations[i][6])))))+1j*((1-2*binary_combinations[i][1])*(8-(1-2*binary_combinations[i][3])*(4-(1-2*binary_combinations[i][5])*(2-(1-2*binary_combinations[i][7])))))))
  return constellation_symbols

In [ ]:
# Função para lidar com a seleção do usuário
def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        global modulation_choice
        modulation_choice = change['new']
        print(f"Escolha do usuário: {modulation_choice}")

# Cria um dropdown com as opções de modulação
dropdown = widgets.Dropdown(
    options=["None", "BPSK", "QPSK", "16QAM", "64QAM", "256QAM"],
    description='Modulação:',
    disabled=False,
)

# Adiciona um observador para lidar com mudanças na seleção
dropdown.observe(on_change)

# Exibe o dropdown
display(dropdown)

Dropdown(description='Modulação:', options=('None', 'BPSK', 'QPSK', '16QAM', '64QAM', '256QAM'), value='None')

Escolha do usuário: QPSK


In [ ]:
# Modulation constelation
constellation = np.array(modulation(modulation_choice)) # Converte o resultado de lista para array
constellation1 = np.array(modulation('BPSK'))
NumOfQamPoints = constellation.size
NumOfQamPoints1 = constellation1.size

# Number of bits per QAM symbol
nBits_per_symbol = int(np.log2(NumOfQamPoints))

## Pilots

In general, the channel can be estimated by using a pilot symbols known to both transmitter and receiver.

In [ ]:
# The pilots symbols can be symbols from constellation
pilotsQAMindexes = np.random.randint(0,NumOfQamPoints1,size=(len(pilots_ofdmSyms_indexes),len(pilot_sub_carrier_indexes)))

Xpilots = constellation1[pilotsQAMindexes]

# TX - QAM mapper

In [ ]:
Nsc = len(sub_carriers_indexes)
NOfdmSymsWithData = len(ofdmSyms_with_data)

# create an array of random number between 0 and 3 (4 is not included)
# Each decimal number represents two bits:
# 0-00
# 1-01
# 2-10
# 3-11
dec_ind = np.random.randint(0,NumOfQamPoints,size=(NOfdmSymsWithData,Nsc))
#bin_input = [np.binary_repr(i, width=nBits_per_symbol) for i in dec_ind]

# QAM Symbols
QAM_symbols = constellation[dec_ind]

#table_data = list(zip(sub_carriers_indexes,dec_ind,, QAM_symbols.astype(str)))

#print(tabulate(table_data, headers=['Subcarrier Index','dec_ind', 'Binary','QAM Symbols (x)']))




# TX - OFDM modulator


In [ ]:
X = np.zeros((NOfdmSyms,N))+1j*np.zeros((NOfdmSyms,N))

# Serial to parallel (Frequency domain representation of the data)
# subcarriers that transport QAM symbols with data
X[ofdmSyms_with_data[:, None],sub_carriers_indexes]=QAM_symbols

# Fill subcarrier in OFDM symbols that carriers pilots
X[pilots_ofdmSyms_indexes[:, None],pilot_sub_carrier_indexes] = Xpilots

# Time domain signal. Each row is a new OFDM symbol without CP
x = np.fft.ifft(X,norm='ortho')

# Cyclic Prefix: Get the last samples of x
cp = x[:,-CP_length:]

# Time domain signal. Each row is a new OFDM symbol with CP
x_with_cp_mtx = np.hstack((cp,x))

power_per_ofdm_symbol_time = np.mean(np.abs(x_with_cp_mtx)**2,axis=1)
power_per_ofdm_symbol_freq = np.mean(np.abs(X)**2,axis=1)

# Parallel to serial.
# The command below gets each row and concatenate with the previous one
# row1, row2, row 3, etc
x_with_cp_serial = x_with_cp_mtx.ravel()


In [ ]:
if 1:
  fig1 = px.bar(y=np.abs(X[0,:]))
  # Update x-axis label
  fig1.update_xaxes(title_text="Subcarrier index")
  # Update y-axis label
  fig1.update_yaxes(title_text="Magnitude of the subcarrier")
  fig1.show()

  # plot the first OFDM signal in time domain (without cyclic prefix)
  fig2 = px.line(y=[x[0,:].real,x[0,:].imag], title="Time domain signal (without Cyclic prefix)", markers=True)
  fig2.data[0].name='Real(x)'
  fig2.data[1].name='Imaginary(x)'
  fig2.show()


  # plot the first OFDM signal in time domain (with cyclic prefix)
  title_msg = 'Time domain signal with Cyclic prefix (CP).'
  fig3 = px.line(x=np.arange(-CP_length,N),y=[x_with_cp_mtx[0,:].real,x_with_cp_mtx[0,:].imag], title=title_msg, markers=True)
  fig3.data[0].name='Real(x)'
  fig3.data[1].name='Imaginary(x)'

  fig3.add_vrect(x0=-CP_length, x1=-1,
              annotation_text="Cyclic prefix", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)

  fig3.show()

  fig4 = go.Figure()
  fig4.add_trace(go.Scatter(y=power_per_ofdm_symbol_time))
  fig4.add_trace(go.Scatter(y=power_per_ofdm_symbol_freq))
   # Update x-axis label
  fig4.update_xaxes(title_text="OFDM symbol index")
  # Update y-axis label
  fig4.update_yaxes(title_text="Average Power")
  fig4.show()

# Configuration of Channel  simulation

In [ ]:
# Channel
#h= np.array([1, 0, 0.1])

# The two lines below creates a channel that only delay the input in 10 samples
# h = np.zeros(16)
# h[10]=1


freqs = np.array((0,1))
gains = np.array((0.01,0.5))

h=firls(3,(0,1,2,3,4,5,6,7,) , (0,0.1,0.2,0.3,0.4,0.5,0.6,0.7), fs=15)
#h=firwin(11, 10e6,fs=Fs)

# Frequency response
w, H = freqz(h, worN=N,whole=True)

if 1:
  fig4 = make_subplots(rows=1, cols=2,subplot_titles=['Magnitude of frequency response', 'Phase of the frequency resposnse'])
  #fig4 = make_subplots(rows=1, cols=2)
  trace1 = go.Scatter(y=np.abs(H),name='abs(H)')
  fig4.add_trace(trace1, row=1,col=1)
  fig4.update_xaxes(title_text="Subcarrier index", row=1,col=1)
  fig4.update_yaxes(title_text="|H|", row=1,col=1)

  #trace2 = go.Scatter(y=np.angle(H)/np.pi, mode='lines', name='Line 2')
  #px.line(y=np.abs(H),markers=True)
  trace2 = go.Scatter(y=np.angle(H)/np.pi,name='phase(H)')
  fig4.add_trace(trace2,row=1,col=2)
  fig4.update_xaxes(title_text="Subcarrier index", row=1,col=2)
  fig4.update_yaxes(title_text="∠H (*pi)", row=1,col=2)

  # Update x-axis label
  #fig4.update_xaxes(title_text="Subcarrier index")
  # Update y-axis label
  #fig4.update_yaxes(title_text="|H|")
  fig4.show()

# Channel

In [ ]:
# Convolve the time domain TX output with the channel impulse response (CIR)
ch_out = np.convolve(x_with_cp_serial, h, mode='full')

# Create complex random noise with power Pnoise
w = np.sqrt(Pnoise/2)*(np.random.randn(ch_out.size)+1j*np.random.randn(ch_out.size))

# Additive white Gaussian noise (AWGN)
y = ch_out+w

In [ ]:
# find the first sample of the OFDM symbol after the delay of the channel
def find_first_sample(pilot,signal):

  # Perform cross-correlation
  correlation = np.correlate(signal,pilot , mode='full')

  normalized_correlation = np.abs(correlation)/max(np.abs(correlation))

  # Generate lags corresponding to the cross-correlation
  lags = np.arange(-(len(pilot)-1), len(signal))

  # Find the lag with maximum correlation
  max_corr_index = np.argmax(normalized_correlation)
  max_corr_lag = lags[max_corr_index]
  max_corr_value = normalized_correlation[max_corr_index]
  #print(len(correlation))
  #print(max_corr_index)

  fig5 = px.line(x=lags,y=normalized_correlation,title="Cross Correlation Between the pilot and channel output", markers=True)
  #plt.text(max_corr_lag,max_corr_value,'(x=%d, y=%.2f)'% (max_corr_lag,max_corr_value))

  fig5.add_annotation(
    text='(x=%d, y=%.2f)'% (max_corr_lag,max_corr_value),
    x=max_corr_lag, y=max_corr_value,  # Coordinates where the text will be placed
    showarrow=False,  # Do not show an arrow pointing to the annotation
    font=dict(size=12, color='black')  # Customize font size and color
  )

  fig5.show()

  return max_corr_lag

pilot0 = x[pilots_ofdmSyms_indexes[0],:]

first_sample = find_first_sample(pilot0,y)
#first_sample = find_first_sample(pilot0,y)

print('First Sample after passing the channel: ',first_sample)

First Sample after passing the channel:  11


# Receiver: OFDM demodulator, channel estimation and frequency equalization

In [ ]:
#Receiver
sym_and_cp_length = N+CP_length

# preinitialize the received symbols for improve loop processing
y_sym_mtx = np.zeros((NOfdmSyms,N))+1j*np.zeros((NOfdmSyms,N))


# You can Skip first samples by setting first_sample higher than zero, if the
# channel has some delay.
# Then, we get NOfdmSyms, taking into accout the lenght
# of the OFDM symbol with cyclic prefix
# This loop start in the first sample of the OFDM symbol
for l in range(NOfdmSyms):
  # For each OFDM Symbol, we skip the CP part and get only N samples
  fromIndex = first_sample+l*(sym_and_cp_length)
  toIndex   = first_sample+l*(sym_and_cp_length)+N
  y_sym_mtx[l,:] = y[fromIndex:toIndex]
  #print('From %d to %d' %(fromIndex,toIndex))

#y_sym_serial = y[first_sample:first_sample+NOfdmSyms*sym_and_cp_length]

#y_sym_withcp_mtx = y_sym_serial.reshape(NOfdmSyms,sym_and_cp_length)

# remove CP
#y_sym_mtx = y_sym_withcp_mtx[:,CP_length:]

# get the signal in frequency domain
Y = np.fft.fft(y_sym_mtx,norm='ortho')

# Channel Estimation
Hest = Y[pilots_ofdmSyms_indexes[:, None],pilot_sub_carrier_indexes]/Xpilots

Hest_mean = np.mean(Hest,axis=0)
#Hest_mean = Hest[0,:]


Yeq = Y/Hest_mean


## plots
if 1:
  fig6 = make_subplots(rows=1, cols=3,
                       subplot_titles=['Magnitude of the Transmitted QAM symbols',
                                       'Magnitude of the received symbols',
                                       'Estimated freq. response of the Channel'])

  # First OFDM symbol on frequency domain
  trace1 = go.Scatter(y=np.abs(X[0,:]))
  fig6.add_trace(trace1,row=1,col=1)
  fig6.update_xaxes(title_text="Subcarrier index", row=1,col=1)

  for index in pilots_ofdmSyms_indexes:
    tmp_legend_str = 'RX OFDM index=%d' % (index)
    tmp_trace = go.Scatter(y=np.abs(Y[index,:]),name=tmp_legend_str)
    fig6.add_trace(tmp_trace,row=1,col=2)
  fig6.update_xaxes(title_text="Subcarrier index", row=1,col=2)

  for index in range(0, len(pilots_ofdmSyms_indexes)):
    tmp_legend_str = 'Hest OFDM symbol=%d' % (index)
    tmp_trace = go.Scatter(y=np.abs(Hest[index,:]),name=tmp_legend_str)
    fig6.add_trace(tmp_trace,row=1,col=3)

  tmp_legend_str = 'Hest mean'
  tmp_trace = go.Scatter(y=np.abs(Hest_mean),name=tmp_legend_str,marker=dict(symbol='x'))
  fig6.add_trace(tmp_trace,row=1,col=3)

  fig6.show()


  fig7 = make_subplots(rows=1, cols=2,
                       subplot_titles=['Before Channel Equalization',

                                       'After Channel Equalization'])

  # set this variable True or False, in order to see QAM points per subcarrier in
  # different colors or all points with the same color
  plot_per_subcarrier = True

  if plot_per_subcarrier:
    # plot constellation per subcarrier
    for index_sc in range(N):
      tmp_legend_str = 'Subcarrier index=%d' % (index_sc)
      tmp_trace = go.Scatter(x=Y[:,index_sc].real, y=Y[:,index_sc].imag,
                             name=tmp_legend_str,mode='markers',
                             legendgroup=index_sc)
      #tmp_trace = go.Scattergl(x=Y[:,index_sc].real, y=Y[:,index_sc].imag, mode='markers')
      fig7.add_trace(tmp_trace,row=1,col=1)
  else:
    # plot constellation for all subcarrier with the same color
      tmp_trace = go.Scatter(x=np.ravel(Y.real), y=np.ravel(Y.imag),name='RX QAM symbols',mode='markers')
      #tmp_trace = go.Scattergl(x=Y[:,index_sc].real, y=Y[:,index_sc].imag, mode='markers')
      fig7.add_trace(tmp_trace,row=1,col=1)

  tmp_trace = go.Scatter(x=constellation[:].real, y=constellation[:].imag,
                         name='Constellation',mode='markers',
                         legendgroup = 'Constellation',
                         marker=dict(symbol='cross',size=14,color='blue'))
  fig7.add_trace(tmp_trace,row=1,col=1)

  #fig7.show()

  # Plot contellation of the data after channel equalization
  if plot_per_subcarrier:
    # plot constellation per subcarrier
    for index_sc in range(N):
      tmp_legend_str = 'Subcarrier index=%d' % (index_sc)
      tmp_trace = go.Scatter(x=Yeq[:,index_sc].real, y=Yeq[:,index_sc].imag,
                             name=tmp_legend_str,mode='markers',
                             legendgroup=index_sc,showlegend=False)
      #tmp_trace = go.Scattergl(x=Y[:,index_sc].real, y=Y[:,index_sc].imag, mode='markers')
      fig7.add_trace(tmp_trace,row=1,col=2)
  else:
      tmp_trace = go.Scatter(x=np.ravel(Yeq.real), y=np.ravel(Yeq.imag),name='RX QAM symbols',mode='markers')
      #tmp_trace = go.Scattergl(x=Y[:,index_sc].real, y=Y[:,index_sc].imag, mode='markers')
      fig7.add_trace(tmp_trace,row=1,col=2)

  tmp_trace = go.Scatter(x=constellation[:].real, y=constellation[:].imag,
                         name='Constellation',mode='markers',
                         legendgroup = 'Constellation',showlegend=False,
                         marker=dict(symbol='cross',size=14,color='blue'))
  fig7.add_trace(tmp_trace,row=1,col=2)

  fig7.show()

## RX QAM demapper

In [ ]:


# Calculate the closest element in constellation for each element in yeq
# The decoded QAM symbol is the closest elemnt in the constellation
closest_elements = [min(constellation, key=lambda x: np.abs(x - ytmp)) for ytmp in Yeq.flatten()]

# Reshape back to the samee shape of Yeq
recoveredQAMSymbols = np.array(closest_elements).reshape(Yeq.shape)

#Compare with the transmitted Symbols X
errors = ~np.equal(recoveredQAMSymbols,X)

error_symbols = Yeq[errors]


# Get the Symbols only for OFDM symbols  and subcarriers with data
# It does not load Resource elements from pilots neither subcarrier that were not used to trnasmit data
Yeq_valid_subcarriers = Yeq[ofdmSyms_with_data[:, None],sub_carriers_indexes]
X_valid_subcarriers   = X[ofdmSyms_with_data[:, None],sub_carriers_indexes]
errors_valid_subcarriers = errors[ofdmSyms_with_data[:, None],sub_carriers_indexes]
error_symbols_valid_subcarriers = Yeq_valid_subcarriers[errors_valid_subcarriers]


fig8 = go.Figure()
#Get only the valid
tmp_trace=go.Scatter(x=np.ravel(Yeq_valid_subcarriers.real),
                     y=np.ravel(Yeq_valid_subcarriers.imag),
                     mode='markers',marker=dict(symbol='x'))
fig8.add_trace(tmp_trace)

tmp_trace = go.Scatter(x=error_symbols_valid_subcarriers.real,
                       y=error_symbols_valid_subcarriers.imag,
                       mode='markers')
fig8.add_trace(tmp_trace)


# plot line between TX and RX equalized QAM symbols that were received with error
if 1:
  for i in range(len(errors_valid_subcarriers)):
      # Iterate over columns
      for j in range(len(errors_valid_subcarriers[i])):
        if errors_valid_subcarriers[i][j]:
          fig8.add_shape(type="line",
                        x0=X_valid_subcarriers[i][j].real,
                        y0=X_valid_subcarriers[i][j].imag,
                        x1=Yeq_valid_subcarriers[i][j].real,
                        y1=Yeq_valid_subcarriers[i][j].imag,
                        line=dict(color="red", width=2))
          #plt.plot([X_valid_subcarriers[i][j].real, Yeq_valid_subcarriers[i][j].real]
          #        ,[X_valid_subcarriers[i][j].imag, Yeq_valid_subcarriers[i][j].imag]
          #        ,linestyle='--',color='r')




# Add a horizontal line at y=0
fig8.add_hline(y=0,line_dash='dot')
fig8.add_vline(x=0,line_dash='dot')


fig8.show()

errors_int_matrix = np.zeros(shape=X.shape,dtype=int)

# plot the erroneous data as a colormap, where red indicates erros ina given
# subcarrier and OFDM symbol
# Convert boolean matrix to integer matrix (0 for False, 1 for True)
tmp = errors.astype(int)+1
#errors_int_matrix = tmp
errors_int_matrix[ofdmSyms_with_data[:, None],sub_carriers_indexes] = tmp[ofdmSyms_with_data[:, None],sub_carriers_indexes]

# pilots are marked as zero
#errors_int_matrix[pilots_ofdmSyms_indexes[:,None],pilot_sub_carrier_indexes]=0



fig = go.Figure(data=go.Heatmap(
        z=errors_int_matrix,
        colorbar = dict(thickness=25,
                                     tickvals=[0,1,2],
                                     ticktext=['0-Not used','1-No error','2-Error'])))

# Update x-axis and y-axis labels
fig.update_xaxes(title_text='Subcarrier index')
fig.update_yaxes(title_text='OFDM Symbol')
# Set the initial title
fig.update_layout(title='OFDM grid. 0 inidcates Resource elements that are not used, 1 inidcates no error and 2 indicates erros.')

fig.update_layout(xaxis_showgrid=True, yaxis_showgrid=True)

fig.show()

In [ ]:
SER = error_symbols_valid_subcarriers.size/(X_valid_subcarriers.size)

# bits transmitted per OFDM symbol
B = Nsc*nBits_per_symbol
Ts = 1/Fs
Tsym = N*Ts
Tcp = CP_length*Ts
T_total = Tsym+Tcp

R = B / T_total

#subcarrier spacing
Deltaf = Fs/N

BWmax = Deltaf*N
BWuse = Deltaf*(sub_carriers_indexes[-1] - sub_carriers_indexes[0])

print("Bits per OFDM Symbol: ", B)
print("Period of an OFDM symbol (IFFT+CP): ", T_total)
print("Data rate (Mbps):", R/1e6)
print("Subcarrier Spacing (kHz): ", Deltaf/1e3)
print("BW (MHz): ",  BWuse/1e6)
print("Number of Errouneuos symbols: ", error_symbols_valid_subcarriers.size)
print("SER:", SER)


Bits per OFDM Symbol:  32
Period of an OFDM symbol (IFFT+CP):  8.463541666666667e-07
Data rate (Mbps): 37.809230769230766
Subcarrier Spacing (kHz):  1920.0
BW (MHz):  28.8
Number of Errouneuos symbols:  30
SER: 0.025


# Assigment

## Problem

1. Read chapters 4, 5 and 6 from the book:
Y. S. Cho, J. Kim, W. Y. Yang, C. G. Kang, MIMO-OFDM Wireless Communications with Matlab, Wiley-IEEE Press, 2010.

2. Create Pilots symbols, in order to estimate channel, based on the received Symbols.

3. Based on the estimated channel, perform channel equalizaiton on the received symbols, calculate the EVM of the QAM symbols that transport data.

4. The EVM can be used to select modulation scheme(QPSK, 16QAM, 64QAM or 256 QAM) of the next OFDM symbols, and get at maximum a BER of 0.01.
You should select a modulation scheme for each subcarrier, according to the EVM.
The relationship between EVM and ber can be found on the paper:
On the Extended Relationships Among EVM, BER and SNR as Performance Metrics(
https://ieeexplore.ieee.org/abstract/document/4178493).

5. Create a matrix with the number of bits per subcarrier and OFDM symbol.
You can assume the receiver knows the modulation order and the respecive constellation chosen by  the transmitter.
Show such matrix in a Heatmap graph.

6. Estimate the data rate in Mbps of the OFDM transmission based on the paramenters of the simulation:
sampling frequency, bits per QAM symbol in each subcarrier, Number of subcarriers, etc. Also take into consideration that the pilots symbols do not transmit data.

7. Calculate the BER and SER.


## Requirements



*   Maximum transmit power 1 W per OFDM symbol.
*   The channel impulse response and level of noise will be given by the professor in another moment.
*   You can transmit at maximum N=1200 subcarriers, but recall that more subcarrier is less power per subcarrier.
*   The Sampling rate should be 30.72 MHz.

Other informations:
The two students that achieves the highest data rate with a maximum BER of 1% gets 10.
The others will be evaluted in terms of their work and simulation, but they can also achieve 10.

The simulations should be shown during the class



1. Leia os capítulos 4, 5 e 6 do livro:
   Y. S. Cho, J. Kim, W. Y. Yang, C. G. Kang, MIMO-OFDM Wireless Communications with Matlab, Wiley-IEEE Press, 2010.

2. Crie símbolos de piloto para estimar o canal com base nos símbolos recebidos.

3. Com base no canal estimado, realize equalização de canal nos símbolos recebidos, calcule o EVM (Erro de Vector Modulação) dos símbolos QAM que transportam dados.

4. O EVM pode ser usado para selecionar o esquema de modulação (QPSK, 16QAM, 64QAM ou 256QAM) dos próximos símbolos OFDM e alcançar no máximo uma BER (Taxa de Erro de Bit) de 0,01. Você deve selecionar um esquema de modulação para cada subportadora de acordo com o EVM. A relação entre EVM e BER pode ser encontrada no artigo: On the Extended Relationships Among EVM, BER and SNR as Performance Metrics (https://ieeexplore.ieee.org/abstract/document/4178493).

5. Crie uma matriz com o número de bits por subportadora e por símbolo OFDM. Você pode assumir que o receptor conhece a ordem de modulação e a constelação escolhida pelo transmissor. Mostre essa matriz em um gráfico de mapa de calor (Heatmap).

6. Estime a taxa de dados em Mbps (megabits por segundo) da transmissão OFDM com base nos parâmetros da simulação: frequência de amostragem, bits por símbolo QAM em cada subportadora, número de subportadoras, etc. Leve em consideração que os símbolos de piloto não transmitem dados.

7. Calcule a BER (Taxa de Erro de Bit) e a SER (Taxa de Erro de Símbolo).

## Requisitos

* Potência máxima de transmissão de 1 W por símbolo OFDM.
* A resposta impulsiva do canal e o nível de ruído serão fornecidos pelo professor em outro momento.
* Você pode transmitir no máximo N=1200 subportadoras, mas lembre-se de que mais subportadoras resultam em menos potência por subportadora.
* A taxa de amostragem deve ser de 30,72 MHz.

Outras informações:
Os dois alunos que alcançarem a maior taxa de dados com uma BER máxima de 1% receberão nota 10. Os demais serão avaliados com base em seu trabalho e simulação, mas também podem obter uma nota 10.

As simulações devem ser apresentadas durante a aula.

